In [101]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from tqdm import tqdm

In [102]:
df = pd.read_csv('clean_data.csv')
df = df.drop(labels=['genre'], axis=1)
df.drop_duplicates(inplace=True)
df = df.groupby(['id', 'name', 'artist']).mean().reset_index()
df.drop_duplicates('id', inplace=True)

In [103]:
def normalize_column(col):
    max_d = df[col].max()
    min_d = df[col].min()
    df[col] = (df[col] - min_d)/(max_d - min_d)

In [104]:
num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num = df.select_dtypes(include=num_types)
        
for col in num.columns:
    normalize_column(col)

In [105]:
km = KMeans(n_clusters=25)
predictions = km.fit_predict(num)
df['cluster'] = predictions
normalize_column('cluster')

In [112]:
num = df.select_dtypes(include=num_types)
columns = ['name', 'artist', 'popularity', 'danceability', 'loudness', 'speechiness', 'acousticness', 'year']
def get_recommendations(song_name, n_top):
        distances = []
        song = df[df.name.str.lower() == song_name.lower()].head(1).values[0]
        rem_data = df[df.name.str.lower() != song_name.lower()]
        for r_song in tqdm(rem_data.values):
            dist = 0
            for col in np.arange(len(rem_data.columns)):
                if not col in [0, 1, 2]:
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
        rem_data['distance'] = distances
        rem_data = rem_data.sort_values('distance')
        return rem_data[columns][:n_top]

get_recommendations('Tommy\'s Party', 10)

100%|██████████| 198338/198338 [00:05<00:00, 35728.61it/s]
C:\Users\khoat\AppData\Local\Temp/ipykernel_39108/1994740248.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rem_data['distance'] = distances


,name,artist,popularity,danceability,loudness,speechiness,acousticness,year
80876,4 Seasons,Rex Orange County,0.660,0.511134,0.817802,0.040538,0.217871,0.966102
179706,Asylum,Chloe Foy,0.590,0.441296,0.788074,0.031334,0.122490,0.983051
30285,we fell in love in october,girl in red,0.670,0.572874,0.740336,0.028956,0.113454,0.983051
131609,Cold Showers,Chelsea Cutler,0.610,0.631579,0.846589,0.033195,0.102410,0.983051
48896,John My Beloved,VÉRITÉ,0.575,0.398785,0.781987,0.047570,0.205823,0.966102
114002,Spirit Lead Me - Live,Influence Music,0.580,0.517206,0.802036,0.031127,0.222892,0.983051
16509,When The World Was At War We Kept Dancing,Lana Del Rey,0.610,0.623482,0.827780,0.035471,0.097390,0.966102
159265,Poetry,Wrabel,0.620,0.581984,0.854716,0.032472,0.020884,0.966102
62457,Wildfire,Seafret,0.620,0.594130,0.817850,0.034023,0.253012,0.949153
110237,Wish You Were Here,Avenged Sevenfold,0.545,0.578947,0.808986,0.030817,0.001777,0.966102


In [113]:
df[df.id == '5OuJTtNve7FxUX82eEBupN'][columns]

,name,artist,popularity,danceability,loudness,speechiness,acousticness,year
137799,Tommy's Party,Peach Pit,0.62,0.571862,0.804436,0.032161,0.12249,0.983051
